# 0. Install Dependencies

In [71]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [72]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [73]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [74]:
env = ShowerEnv()

C:\Users\dima2\venvs\gym-examples\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [75]:
env.observation_space.sample()

array([53.564804], dtype=float32)

In [76]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-26
Episode:2 Score:-44
Episode:3 Score:-2
Episode:4 Score:-26
Episode:5 Score:-40
Episode:6 Score:-60
Episode:7 Score:-50
Episode:8 Score:-26
Episode:9 Score:-24
Episode:10 Score:-36


# 2. Create a Deep Learning Model with Keras

In [77]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [78]:
states = env.observation_space.shape
actions = env.action_space.n

In [79]:
actions

3

In [80]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [82]:
# del model

In [83]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [84]:
model = build_model(states, actions)

In [85]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 24)                48        
                                                                 
 dense_24 (Dense)            (None, 24)                600       
                                                                 
 dense_25 (Dense)            (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [86]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [87]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 25:09 - reward: -1.0000

C:\Users\dima2\venvs\gym-examples\lib\site-packages\keras\engine\training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\dima2\venvs\gym-examples\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 33s 3ms/step - reward: -0.5176
166 episodes - episode_reward: -30.940 [-60.000, 36.000] - loss: 1.105 - mae: 6.499 - mean_q: -9.052

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 33s 3ms/step - reward: -0.4210
167 episodes - episode_reward: -25.461 [-60.000, 46.000] - loss: 2.247 - mae: 10.615 - mean_q: -15.204

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 34s 3ms/step - reward: -0.3626
167 episodes - episode_reward: -21.701 [-60.000, 48.000] - loss: 1.976 - mae: 9.788 - mean_q: -13.971

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 34s 3ms/step - reward: 0.1546
166 episodes - episode_reward: 9.133 [-60.000, 58.000] - loss: 0.854 - mae: 4.804 - mean_q: -4.586

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 34s 3ms/step - reward: 0.7820
done, took 168.069 seconds


In [88]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 58.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 58.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 58.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 58.000, steps: 60
Episode 24: reward: 58.000, steps: 60
Episode 25: reward: 58.000, steps: 60
Episode 26: reward: 60.000, st

In [89]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: ShowerEnv.render() got an unexpected keyword argument 'mode'

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
import gym

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)